# 📖 Analyse automatique des données du site
Ce notebook traite **toutes les sources de données présentes dans le dossier du site**,
en générant :
- des **statistiques descriptives**
- des **boxplots**
- des **histogrammes**
- des **courbes temporelles**

⚙️ Il s'exécute automatiquement via **Papermill** avec un seul paramètre : `site_ref`.

In [ ]:
# 📌 Paramètre Papermill
site_ref = "WFR001_PIOLENC"  # Valeur par défaut pour test local

In [ ]:
# 📌 Import des bibliothèques nécessaires
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("[✅] Bibliothèques importées.")

## 📦 Chargement automatique des données CSV
Le notebook va parcourir le dossier `data/<site_ref>` et charger **tous les fichiers CSV** (hors fichiers `raw_`).

In [ ]:
data_folder = f"data/{site_ref}"
print(f"[📁] Dossier analysé : {data_folder}")

available_files = [f for f in os.listdir(data_folder) if f.endswith(".csv") and not f.startswith("raw_")]
print(f"[✅] Fichiers CSV détectés : {available_files}")

all_data = {}
for filename in available_files:
    path = os.path.join(data_folder, filename)
    try:
        df = pd.read_csv(path)
        if not df.empty:
            key = filename.replace(f"_{site_ref}.csv", "")
            all_data[key] = df
            print(f"[📦] Chargé : {filename} ({len(df)} lignes)")
        else:
            print(f"[⚠️] Fichier vide ignoré : {filename}")
    except Exception as e:
        print(f"[❌] Erreur lecture {filename} : {e}")

print("\n📦 Sources chargées :", list(all_data.keys()))

## 📊 Statistiques descriptives
Calcul des indicateurs statistiques principaux pour toutes les sources chargées.
Le résultat est sauvegardé dans `tables/`.

In [ ]:
stats_list = []

for source, df in all_data.items():
    for col in df.columns:
        if 'wind' in col and df[col].dtype in [np.float64, np.float32, np.int64]:
            series = df[col].dropna()
            if not series.empty:
                stats_list.append({
                    "Source": source,
                    "Variable": col,
                    "Count": series.count(),
                    "Mean": series.mean(),
                    "Std": series.std(),
                    "Min": series.min(),
                    "Max": series.max(),
                    "25%": series.quantile(0.25),
                    "50%": series.quantile(0.50),
                    "75%": series.quantile(0.75)
                })

if stats_list:
    stats_df = pd.DataFrame(stats_list)
    display(stats_df.style.background_gradient(cmap='Blues'))

    # 📌 Sauvegarde CSV
    tables_dir = os.path.join(data_folder, "tables")
    os.makedirs(tables_dir, exist_ok=True)
    stats_path = os.path.join(tables_dir, f"stats_descriptives_{site_ref}.csv")
    stats_df.to_csv(stats_path, index=False)
    print(f"[💾] Statistiques sauvegardées : {stats_path}")
else:
    print("[⚠️] Aucune statistique calculable.")

## 📈 Visualisations automatiques
Création des **boxplots**, **histogrammes** et **courbes temporelles** pour chaque source.
Les images sont sauvegardées dans `figures/`.

In [ ]:
figures_dir = os.path.join(data_folder, "figures")
os.makedirs(figures_dir, exist_ok=True)

for source, df in all_data.items():
    print(f"\n📈 Traitement des graphes pour : {source}")

    wind_cols = [c for c in df.columns if 'wind' in c]

    if wind_cols:
        # Boxplot
        plt.figure(figsize=(10, 6))
        df[wind_cols].boxplot()
        plt.title(f"Boxplot - {source}", fontsize=14)
        plt.ylabel("Vitesse du vent (m/s)", fontsize=12)
        plt.xticks(rotation=45)
        plt.grid()
        path_plot = os.path.join(figures_dir, f"boxplot_{source}.png")
        plt.savefig(path_plot, bbox_inches='tight')
        plt.close()
        print(f"[✅] Boxplot sauvegardé : {path_plot}")

        # Histogrammes
        for col in wind_cols:
            plt.figure(figsize=(10, 6))
            df[col].hist(bins=30, color='steelblue', edgecolor='black')
            plt.title(f"Histogramme - {source} - {col}", fontsize=14)
            plt.xlabel("Valeur", fontsize=12)
            plt.ylabel("Fréquence", fontsize=12)
            plt.grid()
            path_hist = os.path.join(figures_dir, f"hist_{col}_{source}.png")
            plt.savefig(path_hist, bbox_inches='tight')
            plt.close()
            print(f"[✅] Histogramme sauvegardé : {path_hist}")

        # Séries temporelles
        if "date" in df.columns:
            for col in wind_cols:
                plt.figure(figsize=(12, 6))
                plt.plot(pd.to_datetime(df['date']), df[col], color='darkorange')
                plt.title(f"Série temporelle - {source} - {col}", fontsize=14)
                plt.xlabel("Date", fontsize=12)
                plt.ylabel(col, fontsize=12)
                plt.grid()
                path_ts = os.path.join(figures_dir, f"timeseries_{col}_{source}.png")
                plt.savefig(path_ts, bbox_inches='tight')
                plt.close()
                print(f"[✅] Série temporelle sauvegardée : {path_ts}")

## ✅ Fin du traitement
Toutes les statistiques et graphes ont été générés et sauvegardés dans les sous-dossiers appropriés du site.

- 📂 **tables/** pour les statistiques descriptives (CSV)
- 📂 **figures/** pour toutes les images générées (PNG)

Ce notebook peut être exécuté automatiquement via **Papermill** dans votre pipeline.